In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import warnings
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree  import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import BernoulliNB
from lightgbm import LGBMClassifier
from sklearn.feature_selection import RFE
import itertools
from tabulate import tabulate
import os

In [2]:
df = pd.read_csv('/home/sourav/major/Updated_Packet_Trace2.csv', encoding='latin-1')
df

,PACKET_ID,SEGMENT_ID,PACKET_TYPE,CONTROL_PACKET_TYPE/APP_NAME,SOURCE_ID,DESTINATION_ID,TRANSMITTER_ID,RECEIVER_ID,APP_LAYER_ARRIVAL_TIME(ÂµS),TRX_LAYER_ARRIVAL_TIME(ÂµS),...,ACK_NO,isSyn,isAck,isFin,SEGMENT_LEN,SOURCE_IP,DESTINATION_IP,GATEWAY_IP,NEXT_HOP_IP,Malicious/Normal
0,0,NaN,Control_Packet,DSR_RREQ,NODE-9,Broadcast-0,NODE-9,NODE-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,192.168.0.10,255.255.255.255,192.168.0.10,255.255.255.255,Malicious
1,0,NaN,Control_Packet,DSR_RREQ,NODE-9,Broadcast-0,NODE-9,NODE-2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,192.168.0.10,255.255.255.255,192.168.0.10,255.255.255.255,Malicious
2,0,NaN,Control_Packet,DSR_RREQ,NODE-9,Broadcast-0,NODE-9,NODE-3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,192.168.0.10,255.255.255.255,192.168.0.10,255.255.255.255,Malicious
3,0,NaN,Control_Packet,DSR_RREQ,NODE-9,Broadcast-0,NODE-9,NODE-4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,192.168.0.10,255.255.255.255,192.168.0.10,255.255.255.255,Malicious
4,0,NaN,Control_Packet,DSR_RREQ,NODE-9,Broadcast-0,NODE-9,NODE-5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,192.168.0.10,255.255.255.255,192.168.0.10,255.255.255.255,Malicious
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80696,0,NaN,Control_Packet,WLAN_ACK,NODE-17,NODE-1,NODE-17,NODE-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-,-,-,-,Normal
80697,4434,0.0,CBR,App1_CBR,NODE-16,NODE-10,NODE-1,NODE-17,88660000.0,88660000.0,...,NaN,NaN,NaN,NaN,NaN,192.168.0.17,192.168.0.11,192.168.0.2,192.168.0.18,Normal
80698,0,NaN,Control_Packet,WLAN_ACK,NODE-17,NODE-1,NODE-17,NODE-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-,-,-,-,Normal
80699,4432,0.0,CBR,App1_CBR,NODE-16,NODE-10,NODE-17,NODE-10,88620000.0,88620000.0,...,NaN,NaN,NaN,NaN,NaN,192.168.0.17,192.168.0.11,192.168.0.18,192.168.0.11,Normal


In [3]:
from sklearn.preprocessing import LabelEncoder
def le(df):
    for col in df.columns:
        if df[col].dtype == 'object':
                label_encoder = LabelEncoder()
                df[col] = label_encoder.fit_transform(df[col])

le(df)
df

,PACKET_ID,SEGMENT_ID,PACKET_TYPE,CONTROL_PACKET_TYPE/APP_NAME,SOURCE_ID,DESTINATION_ID,TRANSMITTER_ID,RECEIVER_ID,APP_LAYER_ARRIVAL_TIME(ÂµS),TRX_LAYER_ARRIVAL_TIME(ÂµS),...,ACK_NO,isSyn,isAck,isFin,SEGMENT_LEN,SOURCE_IP,DESTINATION_IP,GATEWAY_IP,NEXT_HOP_IP,Malicious/Normal
0,0,NaN,1,8,20,0,20,0,NaN,NaN,...,NaN,2,2,1,NaN,1,22,1,22,0
1,0,NaN,1,8,20,0,20,11,NaN,NaN,...,NaN,2,2,1,NaN,1,22,1,22,0
2,0,NaN,1,8,20,0,20,14,NaN,NaN,...,NaN,2,2,1,NaN,1,22,1,22,0
3,0,NaN,1,8,20,0,20,15,NaN,NaN,...,NaN,2,2,1,NaN,1,22,1,22,0
4,0,NaN,1,8,20,0,20,16,NaN,NaN,...,NaN,2,2,1,NaN,1,22,1,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80696,0,NaN,1,12,8,1,8,0,NaN,NaN,...,NaN,2,2,1,NaN,0,0,0,0,1
80697,4434,0.0,0,2,7,2,0,8,88660000.0,88660000.0,...,NaN,2,2,1,NaN,8,2,11,9,1
80698,0,NaN,1,12,8,1,8,0,NaN,NaN,...,NaN,2,2,1,NaN,0,0,0,0,1
80699,4432,0.0,0,2,7,2,8,1,88620000.0,88620000.0,...,NaN,2,2,1,NaN,8,2,9,2,1


In [8]:
df.columns = df.columns.str.strip()
# df = df.drop([ 'REMOTE_ADDRESS'], axis=1)
# df = df.drop([ 'LOCAL_ADDRESS'], axis=1)
#df = df.drop([ 'MAC_LAYER_ARRIVAL_TIME(µS)'], axis=1)
#df = df.drop([ 'NW_LAYER_ARRIVAL_TIME(µS)'], axis=1)
#df = df.drop([ 'MAC_LAYER_ARRIVAL_TIME(ÂµS)'], axis=1)
df = df.drop([ 'APP_LAYER_ARRIVAL_TIME(ÂµS)'], axis=1)
#df = df.drop([ 'NW_LAYER_ARRIVAL_TIME(ÂµS)'], axis=1)

df

,PACKET_ID,SEGMENT_ID,PACKET_TYPE,CONTROL_PACKET_TYPE/APP_NAME,SOURCE_ID,DESTINATION_ID,TRANSMITTER_ID,RECEIVER_ID,PHY_LAYER_ARRIVAL_TIME(ÂµS),PHY_LAYER_START_TIME(ÂµS),...,ACK_NO,isSyn,isAck,isFin,SEGMENT_LEN,SOURCE_IP,DESTINATION_IP,GATEWAY_IP,NEXT_HOP_IP,Malicious/Normal
0,0,NaN,1,8,20,0,20,0,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
1,0,NaN,1,8,20,0,20,11,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
2,0,NaN,1,8,20,0,20,14,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
3,0,NaN,1,8,20,0,20,15,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
4,0,NaN,1,8,20,0,20,16,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80696,0,NaN,1,12,8,1,8,0,99893651.00,99893806.00,...,NaN,2,2,1,NaN,0,0,0,0,1
80697,4434,0.0,0,2,7,2,0,8,99894059.11,99895054.11,...,NaN,2,2,1,NaN,8,2,11,9,1
80698,0,NaN,1,12,8,1,8,0,99895118.12,99895273.12,...,NaN,2,2,1,NaN,0,0,0,0,1
80699,4432,0.0,0,2,7,2,8,1,99895421.23,99898246.23,...,NaN,2,2,1,NaN,8,2,9,2,1


In [9]:
df

,PACKET_ID,SEGMENT_ID,PACKET_TYPE,CONTROL_PACKET_TYPE/APP_NAME,SOURCE_ID,DESTINATION_ID,TRANSMITTER_ID,RECEIVER_ID,PHY_LAYER_ARRIVAL_TIME(ÂµS),PHY_LAYER_START_TIME(ÂµS),...,ACK_NO,isSyn,isAck,isFin,SEGMENT_LEN,SOURCE_IP,DESTINATION_IP,GATEWAY_IP,NEXT_HOP_IP,Malicious/Normal
0,0,NaN,1,8,20,0,20,0,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
1,0,NaN,1,8,20,0,20,11,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
2,0,NaN,1,8,20,0,20,14,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
3,0,NaN,1,8,20,0,20,15,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
4,0,NaN,1,8,20,0,20,16,100227.00,100691.00,...,NaN,2,2,1,NaN,1,22,1,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80696,0,NaN,1,12,8,1,8,0,99893651.00,99893806.00,...,NaN,2,2,1,NaN,0,0,0,0,1
80697,4434,0.0,0,2,7,2,0,8,99894059.11,99895054.11,...,NaN,2,2,1,NaN,8,2,11,9,1
80698,0,NaN,1,12,8,1,8,0,99895118.12,99895273.12,...,NaN,2,2,1,NaN,0,0,0,0,1
80699,4432,0.0,0,2,7,2,8,1,99895421.23,99898246.23,...,NaN,2,2,1,NaN,8,2,9,2,1


In [10]:
X = df.drop('Malicious/Normal', axis=1)
y = df['Malicious/Normal']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y

0        0
1        0
2        0
3        0
4        0
        ..
80696    1
80697    1
80698    1
80699    1
80700    1
Name: Malicious/Normal, Length: 80701, dtype: int64

In [12]:
rfc = RandomForestClassifier()

rfe = RFE(rfc, n_features_to_select=10)
rfe = rfe.fit(X_train, y_train)

feature_map = [(i, v) for i, v in itertools.zip_longest(rfe.get_support(), X_train.columns)]
selected_features = [v for i, v in feature_map if i==True]

selected_features

['SOURCE_ID',
 'DESTINATION_ID',
 'TRANSMITTER_ID',
 'RECEIVER_ID',
 'PHY_LAYER_ARRIVAL_TIME(ÂµS)',
 'PHY_LAYER_START_TIME(ÂµS)',
 'PHY_LAYER_END_TIME(ÂµS)',
 'PHY_LAYER_PAYLOAD(Bytes)',
 'PACKET_STATUS',
 'SOURCE_IP']

In [13]:
X_train = X_train[selected_features]
# X_train
X_test=X_test[selected_features]
X_test

,SOURCE_ID,DESTINATION_ID,TRANSMITTER_ID,RECEIVER_ID,PHY_LAYER_ARRIVAL_TIME(ÂµS),PHY_LAYER_START_TIME(ÂµS),PHY_LAYER_END_TIME(ÂµS),PHY_LAYER_PAYLOAD(Bytes),PACKET_STATUS,SOURCE_IP
23601,6,12,6,11,12986655.80,12986842.80,12986842.81,60,2,7
62322,7,2,7,14,58240872.62,58241865.62,58241865.63,1540,0,8
23307,19,14,19,13,12932629.25,12932762.25,12932762.26,88,0,21
16370,12,9,9,8,6753195.01,6753460.01,6753460.02,104,0,13
379,18,0,18,8,106736.22,107200.22,107200.23,72,0,20
...,...,...,...,...,...,...,...,...,...,...
54073,12,16,12,15,48402684.03,48402824.03,48402824.04,100,2,13
39691,11,7,11,6,30821762.24,30821917.24,30821917.25,14,2,0
71878,7,2,7,17,75381191.23,75383335.23,75383335.24,1548,0,8
59348,11,7,11,8,55440833.89,55440982.89,55440982.90,116,2,15


In [14]:
import optuna

In [15]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [16]:
def objective(trial):
    n_neighbors = trial.suggest_int('KNN_n_neighbors', 2, 16, log=False)
    classifier_obj = KNeighborsClassifier(n_neighbors=n_neighbors)
    classifier_obj.fit(X_train, y_train)
    accuracy = classifier_obj.score(X_test, y_test)
    return accuracy

In [17]:
study_KNN = optuna.create_study(direction='maximize')
study_KNN.optimize(objective, n_trials=30)
print(study_KNN.best_trial)

[I 2024-04-23 20:25:49,806] A new study created in memory with name: no-name-e623f3fb-2c47-49b2-b496-9fccabd99c80
[I 2024-04-23 20:25:50,769] Trial 0 finished with value: 0.6858311133139211 and parameters: {'KNN_n_neighbors': 4}. Best is trial 0 with value: 0.6858311133139211.
[I 2024-04-23 20:25:51,898] Trial 1 finished with value: 0.7341552567994548 and parameters: {'KNN_n_neighbors': 11}. Best is trial 1 with value: 0.7341552567994548.
[I 2024-04-23 20:25:52,700] Trial 2 finished with value: 0.7341552567994548 and parameters: {'KNN_n_neighbors': 11}. Best is trial 1 with value: 0.7341552567994548.
[I 2024-04-23 20:25:53,443] Trial 3 finished with value: 0.7124713462610743 and parameters: {'KNN_n_neighbors': 8}. Best is trial 1 with value: 0.7341552567994548.
[I 2024-04-23 20:25:54,209] Trial 4 finished with value: 0.7277739916981599 and parameters: {'KNN_n_neighbors': 12}. Best is trial 1 with value: 0.7341552567994548.
[I 2024-04-23 20:25:54,963] Trial 5 finished with value: 0.7324

FrozenTrial(number=18, state=TrialState.COMPLETE, values=[0.7379964066662537], datetime_start=datetime.datetime(2024, 4, 23, 20, 26, 4, 65672), datetime_complete=datetime.datetime(2024, 4, 23, 20, 26, 4, 820042), params={'KNN_n_neighbors': 13}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'KNN_n_neighbors': IntDistribution(high=16, log=False, low=2, step=1)}, trial_id=18, value=None)


In [18]:
KNN_model = KNeighborsClassifier(n_neighbors=study_KNN.best_trial.params['KNN_n_neighbors'])
KNN_model.fit(X_train, y_train)

KNN_train, KNN_test = KNN_model.score(X_train, y_train), KNN_model.score(X_test, y_test)

print(f"Train Score: {KNN_train}")
print(f"Test Score: {KNN_test}")

Train Score: 0.7670384138785625
Test Score: 0.7379964066662537


In [19]:
custom_row = X_test.iloc[[12346]] 
prediction = KNN_model.predict(custom_row)

In [20]:
print('Predicted Target Variable:', prediction[0])

Predicted Target Variable: 0


In [21]:

SEED = 42
knn = KNeighborsClassifier()

In [22]:
from sklearn.model_selection import cross_val_score
models = {}
models['KNeighborsClassifier']= knn

In [23]:
scores = {}
for name in models:
  scores[name]={}
  for scorer in ['precision','recall']:
    scores[name][scorer] = cross_val_score(models[name], X_train, y_train, cv=10, scoring=scorer)

In [24]:
def line(name):
  return '*'*(25-len(name)//2)

for name in models:
  print(line(name), name, 'Model Validation', line(name))

  for scorer in ['precision','recall']:
    mean = round(np.mean(scores[name][scorer])*100,2)
    stdev = round(np.std(scores[name][scorer])*100,2)
    print ("Mean {}:".format(scorer),"\n", mean,"%", "+-",stdev)
    print()

*************** KNeighborsClassifier Model Validation ***************
Mean precision: 
 76.75 % +- 0.34

Mean recall: 
 85.61 % +- 0.47



In [25]:
preds={}
for name in models:
    models[name].fit(X_train, y_train)
    preds[name] = models[name].predict(X_test)
print("Predictions complete.")

Predictions complete.


In [26]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score
def line(name,sym="*"):
    return sym*(25-len(name)//2)
target_names=["normal","anamoly"]
for name in models:
    print(line(name), name, 'Model Testing', line(name))
    print(confusion_matrix(y_test, preds[name]))
    print(line(name,'-'))
    print(classification_report(y_test, preds[name], target_names=target_names))

*************** KNeighborsClassifier Model Testing ***************
[[1772 2908]
 [1589 9872]]
---------------
              precision    recall  f1-score   support

      normal       0.53      0.38      0.44      4680
     anamoly       0.77      0.86      0.81     11461

    accuracy                           0.72     16141
   macro avg       0.65      0.62      0.63     16141
weighted avg       0.70      0.72      0.71     16141



In [27]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [28]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

GaussianNB()

In [29]:

y_pred = nb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Naive Bayes Model Accuracy:', accuracy)

Naive Bayes Model Accuracy: 0.7100551390867976


In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [31]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [32]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [33]:
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Random Forest Model Accuracy:', accuracy)

Random Forest Model Accuracy: 0.9998760919397807


In [34]:
custom_row = X_test.iloc[[616]] 
prediction =rf_model.predict(custom_row)

In [35]:
print('Predicted Target Variable:', prediction[0])

Predicted Target Variable: 1


In [36]:

SEED = 42
rf = RandomForestClassifier()

In [43]:
from sklearn.model_selection import cross_val_score
modelss = {}
modelss['randomforestclassifier']= rf

In [44]:
scores = {}
for name in modelss:
  scores[name]={}
  for scorer in ['precision','recall']:
    scores[name][scorer] = cross_val_score(models[name], X_train, y_train, cv=10, scoring=scorer)

In [45]:
def line(name):
  return '*'*(25-len(name)//2)

for name in modelss:
  print(line(name), name, 'Model Validation', line(name))

  for scorer in ['precision','recall']:
    mean = round(np.mean(scores[name][scorer])*100,2)
    stdev = round(np.std(scores[name][scorer])*100,2)
    print ("Mean {}:".format(scorer),"\n", mean,"%", "+-",stdev)
    print()

************** randomforestclassifier Model Validation **************
Mean precision: 
 100.0 % +- 0.01

Mean recall: 
 99.99 % +- 0.01



In [47]:
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = rf_model.predict(X_test)

In [1]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1-score: {:.2f}".format(f1))

NameError: name 'precision_score' is not defined